In [ ]:
# Hauptfunktion
def main(run_mode, sportid, background_sound, start_date_string, end_date_string, max_download_frequency):
    age_in_hours = get_last_fetch()
    if age_in_hours is not None and age_in_hours > max_download_frequency:
        session = login_with_username_password()
        if session:
            # Alle Aktivitäten im CSV-Format abrufen
            fetch_activity_data_csv(session)
            # Alle neuen Aktivitäten in die Datenbank schreiben
            import_csv_to_sqlite()
    else:
        print(f"Die letzte Abfrage war vor weniger als {max_download_frequency} Stunden oder konnte nicht ermittelt werden.")

    # Dataframe vorbereiten
    activities = query_data_from_db()
    # Introbild vorbereiten/erzeugen
    create_intro_image()
    # Diagramm zeichnen
    create_chart(activities)
    # Video erzeugen
    create_video(activities)
    if use_sound:
        video_with_sound = combine_video_and_audio(final_video_clip, background_sound, sound_fading)
        shutil.move(video_with_sound, final_video_clip) # Erst erzeugen, dann überschreiben, sonst kommt es zu unerwünschtem Standbild
    # Aufräumen
    clean_up_tmp()
    os.system('cls' if os.name == 'nt' else 'clear')
    
if __name__ == "__main__":
    # Argumentparser initialisieren
    parser = argparse.ArgumentParser(description='RUNALYZE2VIDEO')
    # Argumente einsammeln
    parser.add_argument('--run_mode', help='1 letzte Kalenderwoche, 2 letzter Kalendermonat, 3 Benutzerdefinierter Bereich, Start und Ende erforderlich! (Standard: 1)')
    parser.add_argument('--sportid', help='800522 = Laufen / 800524 = Radfahren / 800528 = Wandern (Standard: 800522)')
    parser.add_argument('--background_sound', help='Hintergrundmusik für das Video (Standard: stuff/fleawaltz_fast.mp3)')
    parser.add_argument('--start_date_string', help='Startdatum im Format JJJJ-MM-TT (Standard: 2024-01-01)')
    parser.add_argument('--end_date_string', help='Enddatum im Format JJJJ-MM-TT (Standard: 2024-01-10)')
    parser.add_argument('--max_download_frequency', help='Maximale Downloadfrequenz vom Runalyze JSON-Endpunkt in Stunden (Standard: 23)')

    args = parser.parse_args()
    # Ist alles vorhanden? Wenn nicht, werden Standardwerte genommen.
    if args.run_mode == None:
        run_mode = 1
    if args.start_date_string == None:
        start_date_string = '2024-01-01'
    if args.end_date_string == None:
        end_date_string = '2024-01-10'
    if args.sportid == None:
        sportid = 800522
    if args.background_sound == None:
        background_sound = 'stuff/fleawaltz_fast.mp3'
    if args.max_download_frequency == None:
        max_download_frequency = 23

    # Hauptprogramm mit den übergebenen Arugumenten aufrufen
    main(args.run_mode, args.sportid, args.background_sound, args.start_date_string, args.end_date_string, args.max_download_frequency)